In [0]:
import os, time, requests

HOST = os.environ["DATABRICKS_HOST"]      # 例: https://adb-xxxxxxxxx.y.azuredatabricks.net
TOKEN = os.environ["DATABRICKS_TOKEN"]    # PATやAADトークン
JOB_ID = int(os.environ["JOB_ID"])        # 対象ジョブID
RUN_ID = int(os.environ["RUN_ID"])        # 監視対象のrun_id（スケジュール起動のrunなど）

headers = {"Authorization": f"Bearer {TOKEN}"}

def get_run_state(run_id: int):
    url = f"{HOST}/api/2.1/jobs/runs/get?run_id={run_id}"
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    js = r.json()
    return js["state"]  # {life_cycle_state, result_state, ...}

def pause_job_schedule(job_id: int):
    # 現在のジョブ設定を取得（scheduleをベースに更新）
    get_url = f"{HOST}/api/2.1/jobs/get?job_id={job_id}"
    gj = requests.get(get_url, headers=headers, timeout=30).json()
    schedule = gj["job_settings"].get("schedule")
    if not schedule:
        raise RuntimeError("The job has no schedule configured.")

    schedule["pause_status"] = "PAUSED"

    update_url = f"{HOST}/api/2.1/jobs/update"
    payload = {"job_id": job_id, "new_settings": {"schedule": schedule}}
    ur = requests.post(update_url, headers=headers, json=payload, timeout=30)
    ur.raise_for_status()
    print(f"Job {job_id} schedule paused.")

# 1) run終了を待機
while True:
    state = get_run_state(RUN_ID)
    if state["life_cycle_state"] in ("TERMINATED", "INTERNAL_ERROR"):
        break
    time.sleep(10)

# 2) スケジュールをPAUSEDへ
pause_job_schedule(JOB_ID)